In [1]:
# -*- coding: utf-8 -*-
import warnings

warnings.filterwarnings(action='ignore')
import os, sys
import pandas as pd
import numpy as np
import math

def importOwnLib():
    if '/home/miruware/aProjects/lib' not in sys.path:
        sys.path.append('/home/miruware/aProjects/lib')
        print("lib path is successfully appended.")
    else:
        print("lib path is already exists.")

importOwnLib()
import sonyalib as sonya

import importlib
importlib.reload(sonya)

import datetime
import timeit

from scipy import interp

import keras
from keras import backend as K
from keras.optimizers import Adam
from keras import callbacks, losses
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
# import argparse
# import locale
from keras import initializers

import matplotlib.pyplot as plt
import tensorflow as tf
# %matplotlib inline # jupyter 에서만 사용

## RFE
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import GridSearchCV, StratifiedKFold,train_test_split, KFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, auc, roc_curve
from sklearn.pipeline import Pipeline

from sklearn.metrics import auc, roc_curve  # plot_roc_curve


lib path is successfully appended.


Using TensorFlow backend.


In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True # GPU메모리가 전부 할당되지 않고, 아주 적은 비율만 할당되어 시작해서 프로세스의 메모리 수요에 따라 증가하게 된다.
config.gpu_options.visible_device_list = "0"
K.set_session(tf.Session(config=config))

def mlp_model(num_input = 0, dropout=0, learning_rate=0.005, neuron1=9, neuron2=9):
    global num_features
    num_input = num_features
    keras.backend.clear_session()
    ## 모델 구성하기
    seed_number = 7
    model = Sequential()
    # print learning rate

    model.add(Dense(neuron1, activation='relu', input_dim=num_input, kernel_initializer=initializers.he_normal(seed=seed_number)))
    model.add(BatchNormalization())
    model.add(Dense(neuron2, activation='relu', kernel_initializer=initializers.he_normal(seed=seed_number)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=initializers.he_normal(seed=seed_number)))

    ## 모델 컴파일
    model.compile(optimizer=Adam(learning_rate), loss=losses.binary_crossentropy, metrics=['accuracy'])

    # model.summary()
    return model


def confusion_metrics(conf_matrix):
    # save confusion matrix and slice into four pieces
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    print('True Positives:', TP)
    print('True Negatives:', TN)
    print('False Positives:', FP)
    print('False Negatives:', FN)

    # calculate accuracy
    conf_accuracy = (float(TP + TN) / float(TP + TN + FP + FN))

    # calculate mis-classification
    conf_misclassification = 1 - conf_accuracy

    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    # calculate the specificity
    conf_specificity = (TN / float(TN + FP))

    # calculate precision
    conf_precision = (TN / float(TN + FP))
    # calculate f_1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    print('-' * 50)
    print(f'Accuracy: {round(conf_accuracy, 2)}')
    print(f'Mis-Classification: {round(conf_misclassification, 2)}')
    print(f'Sensitivity: {round(conf_sensitivity, 2)}')
    print(f'Specificity: {round(conf_specificity, 2)}')
    print(f'Precision: {round(conf_precision, 2)}')
    print(f'f_1 Score: {round(conf_f1, 2)}')

    return conf_sensitivity,  conf_specificity



In [8]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    nodes_increment = (last_layer_nodes - first_layer_nodes) / (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

def createmodel(n_layers, first_layer_nodes, last_layer_nodes, learning_rates ,activation_func, loss_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation=activation_func))
    model.compile(optimizer= Adam(learning_rates), loss=loss_func, metrics = ["accuracy"]) #note: metrics could also be 'mse'
    
    return model

##Wrap model into scikit-learn
model =  KerasClassifier(build_fn=createmodel, verbose = False)  

In [12]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    nodes_increment = (last_layer_nodes - first_layer_nodes) / (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

def createmodel(n_layers, first_layer_nodes, last_layer_nodes, learning_rates ,activation_func, loss_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation=activation_func))
    model.compile(optimizer= Adam(learning_rates), loss=loss_func, metrics = ["accuracy"]) #note: metrics could also be 'mse'
    
    return model

##Wrap model into scikit-learn
model =  KerasClassifier(build_fn=createmodel, verbose = False)  

In [43]:
print("This is local file")

start_time = timeit.default_timer()  # 시작 시간 체크

# ==== Step 1. Load original dataset
file_path0 = './BRC_input_201116_train.xlsx'
file_path1 = './BRC_input_201116_test.xlsx'
meta_train = sonya.get_normalized_metadata(file_path0)
meta_test = sonya.get_normalized_metadata(file_path1)
PATH_model = './model_06_NN_grid_sklearn_01'
sonya.createFolder(PATH_model)

properties = [  # total 18
    'sex',
    'age',
    'LR',
    'cT',
    'cN',
    'cAverage',
    'cSD',
    'aAverage',
    'aSD',
    'lMax',
    'homogeneous',
    'hetero',
    'rim',
    'clustered',
    'non-mass',
    'AorCa',
    'LymAo',
    'LymCa'
]

num_properties = len(properties)

X_train = meta_train[properties]
y_train = meta_train['label']

X_test = meta_test[properties]
y_test = meta_test['label']

num_features = len(properties)

batch_size = 25
learning_rate = 0.005
layer1 = 11
layer2 = 11
roc_result = 0

This is local file


In [44]:
activation_funcs = ['relu'] 
loss_funcs = ['binary_crossentropy']
param_grid = dict(n_layers=[2],
                  learning_rates = [0.1, 0.05],
                  first_layer_nodes = [9,11], 
                  last_layer_nodes = [9,11],
                  activation_func = activation_funcs,
                  loss_func = loss_funcs,
                  batch_size = [25],
                  epochs = [20])


In [45]:

for nf in range(num_properties):
    
    num_features = len(properties)
    
    X_train = meta_train[properties]
    y_train = meta_train['label']

    X_test = meta_test[properties]
    y_test = meta_test['label']
    target_dir = PATH_model + '/base_mode_RFE_test00_PR{}'.format(num_features)
    sonya.createFolder(target_dir)
    print('\033[31m \033[43m' + 'Current num of properties: ' + str(num_features) + '\033[0m')
   

    gridCV = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5)
    gridCV.fit(X_train, y_train)

    best_lr = gridCV.best_params_.get('learning_rates')
    best_batch = gridCV.best_params_.get('batch_size')
    best_l1 = gridCV.best_params_.get('first_layer_nodes')
    best_l2 = gridCV.best_params_.get('last_layer_nodes')

    with open(os.path.join(PATH_model, "param_records.txt"), "a") as file:
            param_records = "PR{}\tlr: {}\tbatch: {}\tl1: {}\tl2: {}".format(str(num_features) ,str(best_lr), str(best_batch), str(best_l1), str(best_l2))
            file.write(param_records + "\n")

    best_model = mlp_model(num_input=num_features, learning_rate= best_lr, neuron1=best_l1, neuron2=best_l2)

    kfold = StratifiedKFold(n_splits=10, shuffle=True)

    accuracy = []
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    # K-fold cross validation
    # 학습 데이터를 이용해서 학습

    i = 1
    for train_index, validation_index in kfold.split(X_train,y_train):
        kX_train, kX_test = X_train.iloc[train_index], X_train.iloc[validation_index]
        ky_train, ky_test = y_train.iloc[train_index], y_train.iloc[validation_index]

        cbks = [callbacks.LearningRateScheduler(lambda epoch: 0.001 * 0.5 ** (epoch // 2)),
                callbacks.TensorBoard(write_graph=False)]


        best_model.fit(kX_train, ky_train, epochs=500, batch_size=best_batch, validation_data=(kX_test, ky_test), callbacks=cbks, verbose=0)
        y_val_cat_prob = best_model.predict_proba(kX_test)

        k_accuracy = '%.4f' % (best_model.evaluate(kX_test, ky_test)[1])
        accuracy.append(k_accuracy)

        # roc curve
        fpr, tpr, t = roc_curve(y_train.iloc[validation_index], y_val_cat_prob)
        tprs.append(interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)



        current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        model_fold_name = target_dir + '/'+ str(i) + '_AUC' +str(int(float(roc_auc) * 100)) + "_" + current_time
        model_fold_json = best_model.to_json()

        with open('{}.json'.format(model_fold_name), 'w') as json_file:
            json_file.write(model_fold_json)  # save model per fold

        best_model.save_weights('{}.h5'.format(model_fold_name))  # save weight per fold

        i = i + 1

    with open(os.path.join(PATH_model, "auc_records.txt"), "a") as file:
        auc_records = "PR{}_aucs = ".format(str(num_features)) + str(aucs)
        file.write(auc_records + "\n")

    if num_features == 1: # input 개수가 1 일경우 rfe 진행 x
        with open(os.path.join(PATH_model, "eliminated_features.txt"), "a") as file:
            file.write("{}\t{}\n".format(nf, properties))
        break

    rfe_clf = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=num_features-1)
    rfe_clf.fit(X_train, y_train)
    old_properties = properties
    properties = X_train.columns[rfe_clf.get_support(indices=True)].tolist()

    eliminated_feature = list(set(old_properties) - set(properties))[0]

    with open(os.path.join(PATH_model, "eliminated_features.txt"), "a") as file:
            file.write("{}\t{}\n".format(nf, eliminated_feature))



    print(eliminated_feature)






 Current num of properties: 18
18/18 [==============================] - 0s 118us/step
sex
 Current num of properties: 17
18/18 [==============================] - 0s 114us/step
LymCa
 Current num of properties: 16
18/18 [==============================] - 0s 124us/step
hetero
 Current num of properties: 15
18/18 [==============================] - 0s 122us/step
LR
 Current num of properties: 14
18/18 [==============================] - 0s 128us/step
rim
 Current num of properties: 13
18/18 [==============================] - 0s 126us/step
clustered
 Current num of properties: 12
18/18 [==============================] - 0s 130us/step
non-mass
 Current num of properties: 11
18/18 [==============================] - 0s 114us/step
cAverage
 Current num of properties: 10
18/18 [==============================] - 0s 117us/step
LymAo
 Current num of properties: 9
18/18 [==============================] - 0s 125us/step
aAverage
 Current num of properties: 8
18/18 [==============================] - 0s 

In [37]:
properties

['age',
 'LR',
 'cT',
 'cN',
 'cAverage',
 'cSD',
 'aAverage',
 'aSD',
 'lMax',
 'homogeneous',
 'hetero',
 'rim',
 'clustered',
 'non-mass',
 'AorCa',
 'LymAo',
 'LymCa']

In [38]:
len(properties)

17